# Join two Dataframes by approximate column string matching 
(fussy lookup https://en.wikipedia.org/wiki/Approximate_string_matching)


### This example join two data frames by a common partial column using a Cartesian product and aggregate the result to found with rows doesn't have any match.

First we define two DataFrames df1 and df2 with a common partial column task name and partial task name:

In [1]:
import pandas as pd 

df1 = pd.DataFrame([  
          ['ABC', 'P1']
        , ['BCD', 'P2']
        , ['CDE', 'P3']
       ]
    ,columns = ['task_name', 'pipeline_name']
)

df2 = pd.DataFrame([  
          ['RR', 'C1']
        , ['BC', 'C2']
        , ['HG', 'C3']
        , ['AB', 'C4']
       ]
    ,columns = ['partial_task_name', 'extra_value']
)

Look that AB and BC are partial task of ABC and BCD and can produce ambiguity and many answers

In [2]:
df1

,task_name,pipeline_name
0,ABC,P1
1,BCD,P2
2,CDE,P3


In [3]:
df2

,partial_task_name,extra_value
0,RR,C1
1,BC,C2
2,HG,C3
3,AB,C4


To do a cartesian product or cross Join, we add a join column with the same value in both dataframes and mark every row that found a partial task where partial_task_name is contained in task_name:

In [4]:
df1['join'] = 1
df2['join'] = 1

dfFull = df1.merge(df2, on='join').drop('join', axis=1)
df2.drop('join', axis=1, inplace=True)

dfFull['match'] = dfFull.apply(lambda x: x.task_name.find(x.partial_task_name), axis=1).ge(0)
dfFull

,task_name,pipeline_name,partial_task_name,extra_value,match
0,ABC,P1,RR,C1,False
1,ABC,P1,BC,C2,True
2,ABC,P1,HG,C3,False
3,ABC,P1,AB,C4,True
4,BCD,P2,RR,C1,False
5,BCD,P2,BC,C2,True
6,BCD,P2,HG,C3,False
7,BCD,P2,AB,C4,False
8,CDE,P3,RR,C1,False
9,CDE,P3,BC,C2,False


Then do a group by to aggregate the task and pipeline and we add the column match, when we found at least one match the row is marked as True and if not found any match then is market as False

In [5]:
dfResult = dfFull.groupby(["task_name", "pipeline_name"]).max().reset_index()[['task_name','pipeline_name','match']]
dfResult

,task_name,pipeline_name,match
0,ABC,P1,True
1,BCD,P2,True
2,CDE,P3,False


Then the answer is CDE

In [6]:
dfResult[~dfResult['match']][['task_name','pipeline_name']]

,task_name,pipeline_name
2,CDE,P3


# Using first match will produce a wrong answer

If instead we use only the first match It is going to assign to BC the key task ABC instead of BCD

In [7]:
df2['task_name_match'] = (df2.partial_task_name.apply(lambda x: df1[df1.task_name.str.find(x).ge(0)]['task_name']).iloc[:, 0])
df2

,partial_task_name,extra_value,task_name_match
0,RR,C1,NaN
1,BC,C2,ABC
2,HG,C3,NaN
3,AB,C4,ABC


Then the answer is incorrect because bring also BCD

In [8]:
dfR = df1.merge(df2, left_on='task_name', right_on='task_name_match',how='left')
dfR[dfR['partial_task_name'].isna()][['task_name','pipeline_name']]

,task_name,pipeline_name
2,BCD,P2
3,CDE,P3


Because BC was assigned to ABC instead of BCD:

In [9]:
dfR

,task_name,pipeline_name,join,partial_task_name,extra_value,task_name_match
0,ABC,P1,1,BC,C2,ABC
1,ABC,P1,1,AB,C4,ABC
2,BCD,P2,1,NaN,NaN,NaN
3,CDE,P3,1,NaN,NaN,NaN
